In [104]:
# Assemble the input files for SPECFEM3D

from os.path import join, dirname
import numpy as np
from model_utils import vp2rho
import pandas as pd


In [105]:

inpath_para = "/Volumes/OmanData/geophones_no_prefilt/data/specfem_model_para.dat"
inpath_p = "/Volumes/OmanData/geophones_no_prefilt/data/shallow_vp_profile.dat"
inpath_s = "/Volumes/OmanData/geophones_no_prefilt/data/shallow_vs_profile.dat"

q_kappa = 2444.4
q_mu = 20.0

numx = 104
numy = 104

In [106]:
# Read the SPECFEM parameter file
paradf = pd.read_csv(inpath_para, sep=" ")
numel = paradf["number_elements"].sum()

In [107]:
## Read the input models
depax_p, pvels_in = np.loadtxt(inpath_p, unpack=True)
depax_s, svels_in = np.loadtxt(inpath_s, unpack=True)

In [108]:
## Interpolate the models to the desired depth axis
depth_top = 0.0
list_ma = []
for lyrind, row in paradf.iterrows():
    numel_lyr = int(row["number_elements"])
    depth_bot = row["depth"]

    ddep = (depth_bot - depth_top) / numel_lyr
    depax = np.linspace(depth_top, depth_bot, numel_lyr)

    pvels = np.interp(depax, depax_p, pvels_in)
    svels = np.interp(depax, depax_s, svels_in)
    rhos = vp2rho(pvels / 1000.0)*1000.0
    q_kappas = np.ones(numel_lyr) * q_kappa
    q_mus = np.ones(numel_lyr) * q_mu
    aniflags = np.zeros(numel_lyr)
    domainflags = 2*np.ones(numel_lyr)

    output_ma = np.column_stack((rhos, pvels, svels, q_kappas, q_mus, aniflags, domainflags))
    list_ma.append(output_ma)

    depth_top = depth_bot

output_ma = np.vstack(list_ma)
numma = output_ma.shape[0]
mainds = np.arange(1, numma+1)
output_ma = np.column_stack((mainds, output_ma))

elinds = np.arange(1, numel+1)
elinds = elinds[::-1]

mainds = np.arange(1, numel+1)


In [109]:
## Save to file
outdir = dirname(inpath_p)

outpath = join(outdir, "specfem_input_model.dat")
output_mod = np.stack([np.ones(numma), np.ones(numma)*numx, np.ones(numma), np.ones(numma)*numy, elinds, elinds, mainds], axis=1)
np.savetxt(outpath, output_mod, fmt="%d %d %d %d %d %d %d")

outpath = join(outdir, "specfem_input_materials.dat")
np.savetxt(outpath, output_ma, fmt="%d %.1f %.1f %.1f %.1f %.1f %d %d")
